In [1]:
import unicodedata
import pandas as pd
from epiweeks import Week
import numpy as np
import geopandas as gpd
from shapely.geometry import Point


# Subsampling scheme for genomic database
In order to limit the computational burden of the phylogeographic analysis, we subsampled 2500 genomes from our genomic dataset. Our subsampling scheme allocates sequences to locations based on their proximity and estimated air travel to San Diego County. Once allocated, sequences are randomly sampled proportional to the location-specific incidence data binned by epidemiological week.

We first load in the incidence data, which was downloaded and formatted by the `download-format-cases.R`. Epiweek needs to be recalculated because the `floor_date` function of the `lubridate` package generates different results from the `Week` function of the `epiweeks` library.

In [2]:
total = pd.read_csv( "cases.csv", usecols=["name", "week", "new_cases"] )
total["week"] = pd.to_datetime( total["week"] )
total["week"] = total["week"].apply( lambda x: Week.fromdate(x).startdate() )
total.head()

,name,week,new_cases
0,Alabama,2020-02-09,0
1,Alabama,2020-02-16,0
2,Alabama,2020-02-23,0
3,Alabama,2020-03-01,0
4,Alabama,2020-03-08,15


Next, we load the genomic database and assign subsampling locations to each sequence. This location is state-level in Canada, Mexico, and the US, and country-level everywhere else. To prevent limited temporal sampling from biasing our analyses, we also limit our dataset to sequences collected before the last Baja California.

In [3]:
usecols = ["strain", "gisaid_accession", "date_collected", "region", "country", "division", "location", "host", "pangolin_lineage"]

seqs = pd.read_csv( "/Users/natem/Dropbox (Scripps Research)/Personal/Code/Projects/project_2021_california-hcov-genomics/data/metadata_filtered.tsv", sep="\t", parse_dates=["date_collected"], usecols=usecols )
seqs = seqs.loc[(~seqs["pangolin_lineage"].isna())&(seqs["host"]=="Human")]
seqs["epiweek"] = seqs["date_collected"].apply( lambda x: Week.fromdate(x).startdate() )

seqs["ss_location"] = seqs["country"]
seqs.loc[seqs["ss_location"].isin( ["USA", "Mexico", "Canada"] ),"ss_location"] = seqs["division"]
seqs.loc[seqs["location"].isin( ["San Diego", "San Diego County"] ), "ss_location"] = "San Diego"
seqs.loc[seqs["location"].isin( ["Los Angeles", "Los Angeles County"] ), "ss_location"] = "Los Angeles"
seqs["ss_location"] = seqs["ss_location"].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())

max_bc = seqs.loc[seqs["ss_location"]=="Baja California","date_collected"].max()
seqs = seqs.loc[seqs["date_collected"]<max_bc]

seqs.head()

,strain,gisaid_accession,date_collected,region,country,division,location,host,pangolin_lineage,epiweek,ss_location
0,AFG|Afghanistan|EPI_ISL_1034760|2020-06-06,EPI_ISL_1034760,2020-06-06,Asia,Afghanistan,Afghanistan,None,Human,B.1.1,2020-05-31,Afghanistan
1,AFG|Afghanistan|EPI_ISL_1000998|2020-06-13,EPI_ISL_1000998,2020-06-13,Asia,Afghanistan,Afghanistan,None,Human,B.1.36,2020-06-07,Afghanistan
2,AFG|Afghanistan|EPI_ISL_1000999|2020-06-07,EPI_ISL_1000999,2020-06-07,Asia,Afghanistan,Afghanistan,None,Human,B.1.9,2020-06-07,Afghanistan
3,AFG|Afghanistan|EPI_ISL_1001000|2020-06-02,EPI_ISL_1001000,2020-06-02,Asia,Afghanistan,Afghanistan,None,Human,B.1,2020-05-31,Afghanistan
4,AFG|Afghanistan|EPI_ISL_1001001|2020-05-30,EPI_ISL_1001001,2020-05-30,Asia,Afghanistan,Afghanistan,None,Human,B.1,2020-05-24,Afghanistan


We left-merge the cases onto the sequences using the epiweek and subsampling location as keys. We then convert these values to sampling weights and correct the weight for the number of sequences collected each epiweek.

It should also be noted that the location and epiweek of some sequences are not present in the case data, but that these represent a relative small fraction of the entire genomic dataset.

In [4]:
seqs_merged = seqs.merge( total, left_on=["ss_location", "epiweek"], right_on=["name", "week"], how="left" )
seqs_merged = seqs_merged.loc[~seqs_merged["new_cases"].isna()]

seqs_merged.loc[seqs_merged["new_cases"]<0,"new_cases"] = 0
seqs_merged["corrected"] = seqs_merged.groupby( ["location", "epiweek"] )["new_cases"].transform( lambda x: x / len( x ) )
seqs_merged["probability"] = seqs_merged["new_cases"] / seqs_merged["new_cases"].sum()

print( seqs.shape )
print( seqs_merged.shape )

(2487539, 11)
(2318356, 16)


We will next allocate sequences to sample from each location. This will be a compound weight of the proximity of each location to San Diego, and the number of flights destined for San Diego from each location in 2019.

Flight data is from the OpenSky Network and is formatted using the `calculate-flights.py` script.

In [12]:
flights = pd.read_csv( "flights.csv" )
flights.head()

,origin,flights
0,Alabama,17
1,Alaska,9
2,Alberta,182
3,Arizona,6089
4,Arkansas,9


Distance is calculated from a shapefile of first administration regions download from https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-1-states-provinces/. Again, we calculate distances at the appropriate location levels.

In [13]:
admin1 = gpd.read_file( "/Users/natem/Documents/Data/shapefiles/ne_10m_admin_1_states_provinces/ne_10m_admin_1_states_provinces.shp" )
admin1 = admin1[["name", "admin", "geometry"]]

us_mx_ca = admin1.loc[admin1["admin"].isin( ["United States of America", "Mexico", "Canada"])]

admin = admin1.loc[~admin1["admin"].isin( ["United States of America", "Mexico", "Canada", "Antarctica"]),["admin", "geometry"]].dissolve( "admin" ).reset_index()
admin.loc[admin["admin"]=="Hong Kong S.A.R.","admin"]= "Hong Kong"
admin.loc[admin["admin"]=="Georgia","admin"] = "GeorgiaC"
admin["name"] = admin["admin"]

print( f"Admin dataframe has shape: {admin.shape}" )
print( f"US Mexico dataframe has shape: {us_mx_ca.shape}" )

admin = pd.concat( [admin, us_mx_ca] )

print( f"Concatenated dataframe has shape: {admin.shape}" )

# Need to convert to a flat surface so distance can be calculated. I'm using epsg2295 which is a mercator projection,
# distance is in meters, probably, but units don't really matter.
admin = admin.to_crs( epsg=3395 )
sd_centroid = Point( -13001670.98, 3874670.85 )
admin["distance"] = admin.distance( sd_centroid )

admin = admin.loc[~admin["name"].isna()]
admin["name"] = admin["name"].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
admin = admin.drop( columns=["geometry"] )
admin["name"] = admin["name"].replace( { "Mexico":"Estado de Mexico", "Distrito Federal" : "Mexico City" } )

Admin dataframe has shape: (249, 3)
US Mexico dataframe has shape: (97, 3)
Concatenated dataframe has shape: (346, 3)


We merge flight and distance data here. To calculate weights, both data sources are normalized to have unit range. Distance data is inverted, so that closer locations have greater weight, and then taken to the 8th power, to provide relatively greater weight to closer locations. These two weights are summed, and their fraction in relation to all locations weights, are the fraction of sequences that are allocated to the location (a floor of 1 sequence is used).

In [31]:
merged = admin.merge( flights, left_on="name", right_on="origin", how="outer" )
merged = merged.loc[~merged["distance"].isna()]
merged.loc[merged["flights"].isna(),"flights"] = 0

# restrict distances to places which have sequences
merged = merged.loc[merged["name"].isin( seqs_merged["ss_location"].unique() )]

merged["normflights"] = merged["flights"] / merged["flights"].max()
merged["normdistance"] = 1 - ( merged["distance"] / merged["distance"].max() )
merged["normdistance"] = np.power( merged["normdistance"], 8 )
merged["sequences"] = merged["normflights"] + merged["normdistance"]
merged["sequences"] = np.round( ( merged["sequences"] / merged["sequences"].sum() ) * 1000 )

min_value = merged.loc[merged["sequences"]>0,"sequences"].min()

seq_dict = merged.set_index( "name" )["sequences"].to_dict()

Here we perform subsampling using the weights and allocations calculated above. 400 sequences are selected each from San Diego County, Los Angeles County, Baja California, and Quebec, Canada. Contextual sequences are sampled by iterating through each other location and sampling the allocated number of sequences. Sequences are given a sampling weight proportional to the incidence of COVID during the epidemiological week in which they were collected.

To support the root of the phylogeny we also include the 50 earliest sequences and the 50 earliest European sequences in the dataset. Lastly, to assess the temporal reconstruction of the phylogeny, we also include genomes from three outbreaks with well-described introductions.



In [34]:
seqs_merged.loc[seqs_merged["ss_location"].isin(["Baja California", "San Diego", "Los Angeles", "Quebec"]),"ss_location"]

San Diego          15558
Quebec             14185
Los Angeles         5744
Baja California     1251
Name: ss_location, dtype: int64

In [45]:
focus = ["Baja California", "San Diego", "Los Angeles", "Quebec"]

specified = seqs_merged.loc[seqs_merged["ss_location"].isin( focus )]
contextual = seqs_merged.loc[~seqs_merged["ss_location"].isin( focus )]

# Contextual
selected_df = list()
for name, df in contextual.groupby( "ss_location" ):
    try:
        requested = int( seq_dict[name] )
    except KeyError:
        #print( f"no entry for {name}. Skipping..." )
        requested = 0

    requested = min( requested, len( df ) )
    try:
        sampled = df.sample( n=requested, replace=False, weights="corrected" )
        sampled["role"] = "contextual"
        selected_df.append( sampled )
    except ValueError:
        continue

# Focal
for name, df in specified.groupby( "ss_location" ):
    sampled = df.sample( n=400, replace=False, weights="corrected" )
    sampled["role"] = name
    selected_df.append( sampled )

# Earliest
# sampled = seqs_merged.sort_values("date_collected").head(50)
# sampled["role"] = "earliest"
# selected_df.append( sampled )

# Earliest European
with open( "worobey_early.txt", "r" ) as f:
    early = [line.strip() for line in f]
sampled = seqs_merged.loc[seqs_merged["gisaid_accession"].isin( early )]
sampled["role"] = "earliest_european"
selected_df.append( sampled )

# Known outbreaks
with open( "known_clades.txt", "r" ) as f:
    clades = [line.strip() for line in f if not line.startswith( "#" )]
sampled = seqs_merged.loc[seqs_merged["gisaid_accession"].isin( clades )]
sampled["role"] = "known_clades"
selected_df.append( sampled )

selected_df = pd.concat( selected_df )
print( f"{selected_df['week'].min()} - {selected_df['week'].max()}")

print( len( selected_df ) )
selected_df.to_csv( "selected_2022-06-23.csv" )

/var/folders/ct/hf5sjtrx29q3hzh3tqrd53740000gn/T/ipykernel_99209/3455392776.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled["role"] = "earliest_european"
/var/folders/ct/hf5sjtrx29q3hzh3tqrd53740000gn/T/ipykernel_99209/3455392776.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled["role"] = "known_clades"


2020-02-09 - 2021-07-11
2647
